#### CIE4604 Simulation and Visualization
# Module 2 Satellite Orbits - Exercise 6 (Simple KML)

**Hans van der Marel, 15 November 2020**

In this notebook we will use the simplekml Python module to visualize satellite orbits in Google Earth.

## Simplekml getting started

From the simplekml homepage https://simplekml.readthedocs.io/en/latest/

> The python package simplekml was created to generate kml (or kmz). It was designed to alleviate the burden of having to study KML in order to achieve anything worthwhile with it. If you have a simple understanding of the structure of KML, then  simplekml is easy to run with and create usable KML.

For an introduction and reference of the Keyhole Markup Language (KML) see the Google pages on KML https://developers.google.com/kml.

Here is an quick example from the simplekml home page to get started:

In [ ]:
import simplekml
kml = simplekml.Kml()
kml.newpoint(name="Kirstenbosch", coords=[(18.432314,-33.988862)])  # lon, lat, optional height
kml.save("botanicalgarden.kml")

View the kml file in **Google Earth** to check if it worked.

If it gives an <span style="color:red">*ModuleNotFoundError*</span> then you have to install simple kml using conda or pip. To install this package with conda run one of the following 
```
conda install simplekml
conda install -c conda-forge simplekml
```
from the Anaconda Prompt. 

After that rerun the above example to see if it works.

Taking our quick example apart, everything we do with simplekml starts with creating an instance of the *simplekml.Kml* class:
```
kml = simplekml.Kml()
```
All arguments passed to the class on creation are the same as that of a simplekml.Document. To change any properties after creation you can do so through the *simplekml.Kml.document()* property:
```
kml.document.name = "Test"
```
To create a fully fledged KML document a document tree is created via calls to various functions like:
```
kml.newpoint()
```
These calls build up a relationships between classes which will be converted into KML when a call to one of the following
```
kml.save()
kml.savekmz()
kml.kml()
```
is made. In the quick example we use *kml.save()* to create the kml file. To print the result to screen you can do
```
print(kml.kml())
```
Let's do that ...

In [ ]:
print(kml.kml())

As you can see, only the least amount of KML is generated for it to be viewable in Google Earth, in case of our quick example just the *name* and *coord* properties of a point which were passed on creation of the point.

Properties can be added or changed later if we do the following

In [ ]:
kml = simplekml.Kml()
pnt = kml.newpoint(name="Kirstenbosch", coords=[(18.432314,-33.988862)])  # lon, lat, optional height
print(kml.kml())

This is the same code as our quick example, with one minor difference, `pnt = kml.newpoint...` . Properties can be added and Attributes can be changed. Here are a few examples...

In [ ]:
pnt.description="Botanical Garden at Kirstenbosch, open weekdays."
pnt.name="Kirstenbosch (SA)"
print(kml.kml())

To add a linestring, a connected set of line segments, do

In [ ]:
lin = kml.newlinestring(name="Pathway", description="A pathway in Kirstenbosch",
                        coords=[(18.43312,-33.98924), (18.43224,-33.98914),
                                (18.43144,-33.98911), (18.43095,-33.98904)])
print(kml.kml())
kml.save("botanicalgarden.kml")

We save the file again, so you can inspect the result in Google Earth.

Polygons, images overlays, etc. work in more or less the same way. See the documentation at https://simplekml.readthedocs.io/en/latest/ for examples and as reference.

## Visualizing satellite orbits in Google Earth

We are going to plot Radarsat-2 orbit tracks in Google Earth. For this we have to load the `tleplot.py` and `crsutil.py` modules, read two-line elements that were previously downloaded, and compute the satellite positions in ECEF for a period of interest. 

In [ ]:
import numpy as np
import tleplot as tle
import crsutil as crs

In [ ]:
# Read TLE from file (we use an existing file, so that we have prior knowledge about the dates)
tleERS = tle.tleread('resource-10-oct-2017.tle', verbose=0)

# Compute satellite positions in ECI
t = tle.tledatenum(['2017-09-28 05:50:00',20,1])
xsat, vsat = tle.tle2vec(tleERS, t, 'RADARSAT-2')

# Convert ECI coordinate to ECEF
xsate, vsate = crs.eci2ecef(t, xsat, vsat)

# Compute lookangles and range-rate (range-rate is part of "lookangles")
#lookangles, flags = crs.satlookanglesp(t, np.hstack([xsate, vsate]), xobje, verbose=1)

Next we have to convert the Cartesian coordinates of the satellite to latitude (deg), longitude (deg) and height (m) for display in KML. For this common operation we write a simple function first, which we are going to call `xyz2coords`.

Note that this function returns a list of (longitude, latitude, height) tuples which are used throughout the simpleKML module. This is somewhat different from other conventions, notably, for KML the longitude is given as the first coordinate. 
Therefore, we decided to split this function up in two other functions: `xyz2llh` which converts the Cartesian coordinates to latitude, longitude and height (in that order!), and `llh2coords` which forms the list of tuples *coords* needed for `simpleKML`.

In [ ]:
def xyz2coords(x):
    """Compute list of (longitude, latitude, height) tuples from Cartesian coordinates.
    """
    lat, lon, height = xyz2llh(x)
    coords = llh2coords(lat, lon, height)
    
    return coords

def xyz2llh(x):
    """Compute latitude, longitude and height from Cartesian coordinates.

    lat, lon, height = xyz2llh(x) computes the spherical latitude [deg], longitude [deg],
    and height [m] from the array Cartesian coordinates X,Y,Z in x [m]. The cartesian 
    coordinates are given in an 2D numpy array x with X, Y and Z in the respective columns.
    """

    Re = 6378136

    R = np.sqrt(np.sum(x**2, axis=1))

    lat = np.arcsin(x[:,2]/R) * 180/np.pi
    lon = np.arctan2(x[:,1], x[:,0]) * 180/np.pi
    height = R-Re
    
    return lat, lon, height

def llh2coords(lat, lon, height):

    if type(lat) in [int, float]:
        lat = [lat]
        lon = [lon]
        height= [height]
        
    coords = []
    for k in range(len(lon)):
         coords.append((lon[k], lat[k], height[k]))
    
    return coords

Let's try this code out using some simple examples.

In [ ]:
lat = [ 52, 53, 54]
lon = [ 4 , 5 ,6]
height = [0, 1, -1]
print(llh2coords(lat,lon,height))

lat = np.array(lat)
lon = np.array(lon)
height = np.array(height)
print(llh2coords(lat,lon,height))

print(llh2coords(52., 4., .5))

It works, let's do something more useful: we convert the Cartesian satellite coordinates in ECEF to a list of (lon, lat, height) tuples. 

We do this with a one-liner, the "\*" means that we pass the output of xyz2llh to llh2coords by reference. This gives the same as using our wrapper function `xyz2coords`. 

In [ ]:
coords = llh2coords( *xyz2llh(xsate) )
print(coords)

Try the wrapper function `xyz2coords` out yourself and check if it gives the same result.

In [ ]:
coords = xyz2coords(xsate)
print(coords)

## Our first KML file with Radarsat-2 orbits

Let's make our first KML file with Radarsat-2 orbits. We also plot a marker at the campus in Delft. The code is actually quite simple...

We save the KML to file, but also we print it to show what is in the file, but the idea is of course to view it in Google Earth.

In [ ]:
coords = xyz2coords(xsate)

kml = simplekml.Kml()
pnt = kml.newpoint(name="Delft", coords=[(4.374, 52.0, 0.0)])  # lon, lat, optional height
pnt.description="The blue line, at Delft University of Technology campus."
lin = kml.newlinestring(name="Radarsat-2", description="Radarsat-2 track",
                        coords=coords)
kml.save("Radarsat2-1.kml")
print(kml.kml())

Open Google Earth with the kml file you just created.

The result is not completely as expected, apparently the Radarsat-2 track is "clamped to the ground". Nice for plotting the ground tracks, but not as intended.
Check this by going to places, right click on "Radarsat-2", and check out the Altitude properties. Is it indeed "clamped to the ground"? Try changing this in Google Earth to one of the other options. Which one do you have to use?

You find in the same tab another option "Extend path to the ground". Try this out as well.

The idea is nice to extend it to the ground, but the "filling" is not. This can be turned off by going to the "Style, color" tab, and select "Outline". Try this out. Another option is to change the opacity of the fill. 

There are a lot of style elements you can adjust. The general idea it to experiment until you are happy with the visual appearance, and then modify our code to obtain the same visual appearance programmatically.

In the next cell we do some of these modification. Please feel free to experiment with this yourself...

## Change styling and visual appearance

There are a lot of options in `simpleKML` to adjust the styling. We are not going to discuss them. For this consult the documentation at https://simplekml.readthedocs.io/en/latest/.

In the cell below we show how to do some of the changes we did before manually in Google Earth. We have also added the sub-satellite track (ground track) to the kml.

In [ ]:
kml = simplekml.Kml()
pnt = kml.newpoint(name="Delft", coords=[(4.374, 52.0, 0.0)])  # lon, lat, optional height
pnt.description="The blue line, at Delft University of Technology campus."
# satellite track
sat = kml.newlinestring(name="Radarsat-2", description="Radarsat-2 track",
                        coords=coords)
sat.altitudemode = simplekml.AltitudeMode.absolute
sat.extrude = 1 
#lin.style.polystyle.fill = 0
sat.style.polystyle.color = "4dffffff"
# sub satellite track
sub = kml.newlinestring(name="Radarsat-2 sub-satellite", description="Radarsat-2 sub-satellite (ground) track",
                        coords=coords)
sub.altitudemode = simplekml.AltitudeMode.clamptoground
sub.style.linestyle.color = simplekml.Color.red
kml.save("Radarsat2-2.kml")

I suggest you try out some other options as well yourself.

## Add timeslider to the KML

In the final example we are going to add a timeslider to the KML. The idea is to show satellite links (as lines) between the satllite and our ground station in Delft.

This is done by adding a new linestring for each link with a timestamp. First we copy the code from our previous example, remove the extrusion and the sub-satellite track, and then add a loop for adding the satellite links.

Note that we added a KML folder to contain all the links (why we should do this?).

In [ ]:
Delft = (4.374, 52.0, 0.0)

kml = simplekml.Kml()
pnt = kml.newpoint(name="Delft", coords=[ Delft ])  # lon, lat, optional height
pnt.description="The blue line, at Delft University of Technology campus."
# satellite track (always visible)
sat = kml.newlinestring(name="Radarsat-2", description="Radarsat-2 track",
                        coords=coords)
sat.altitudemode = simplekml.AltitudeMode.absolute
sat.extrude = 0
# loop to add the satellite links with a time-stamp
fol = kml.newfolder(name='Radarsat-2 links')
for k, coord in enumerate(coords):
    isodate = crs.num2datetime(t[k]).isoformat()
    link = fol.newlinestring(name=isodate, coords=[ coord, Delft])
    link.timestamp.when = isodate
    link.name = isodate
    link.altitudemode = simplekml.AltitudeMode.absolute
    link.extrude = 0
    link.style.linestyle.color = simplekml.Color.red

kml.save("Radarsat2-3.kml")

Did you notice that some of the links go through the Earth? 

Especially for longer simulations we might not want to display the link only when the satellite is visible from the ground station. This can be done using the output from `satlookanglesp` and adding an extra test inside the loop to only visualize links above a certain elevation cutoff.

We leave this as an exercise to you.

In [ ]:
# Space for your own additions ... (and you may create more cells of course)

## Exercise - show Sentinel-1A orbits in Google Earth Pro

In this exercise you have to create a KMZ(KML) file to display Sentinel 1 orbits in Google Earth Pro.

You may use this notebook as starting point. Either create a new notebook, or add your code to this one. Creating a new notebook is probably the cleanest approach. 

This is what you need to do:

- plot Sentinel 1A orbits for 25 November 2020 in Google Earth (select a period yourself, or use different periods)
- plot in Google Earth the Sentinel 1A orbits and observations from Delft for the same day, select only observations above 30 degrees elevation, and Right-looking (see also Exercise 5)

This task requires the `tleplot.py`, `crsutil.py` and `simplekml` modules. You re-use some material from this notebook, but you may also have a look at `CIE4604_M2_Exercise5.ipynb` to reuse some material

\[End of this Jupyter notebook\]